In [1]:
import h5py
import sys
sys.path.append('/Users/zhengj10/Python/Modules')
import os
import numpy as np
from skimage.io import imsave
import matplotlib.pyplot as plt
import yaml
import pandas as pd
import PatcherBot as pb
from glob import glob
import seaborn as sns


In [3]:
# sumPath = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/summary/Voltron2_paper'
# imgPath = '/Volumes/genie/BenArthur/patch-screen/bot'
# rawPath = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw'

sumPath = '/Users/zhengj10/Desktop/GENIE/Data/PatcherBot/summary'
imgPath = '/Volumes/genie/BenArthur/patch-screen/HCImage'
rawPath = '/Volumes/genie/Jihong_GENIE_stuff/Voltron_Kinetics_manual_patch_with_HCImage'

os.path.exists(rawPath)


True

In [5]:
df = pd.read_pickle('/Users/zhengj10/Desktop/GENIE/Manuscript/Voltron 2 paper/Voltron 2 paper_v7/kinetics.pkl')
df

,Date,Cell,Sensor,tau_f_act,tau_s_act,tau_f_act_percent,tau_f_deact,tau_s_deact,tau_f_deact_percent
0,201027,Cell_1,Voltron,0.955512,4.032292,0.581620,1.161170,11.063581,0.587017
1,201027,Cell_2,Voltron,0.767470,2.468679,0.554365,1.261434,4.724830,0.605563
2,201027,Cell_3,Voltron,0.987813,4.335913,0.640978,1.242117,5.448495,0.730667
3,201027,Cell_4,Voltron,0.690084,2.192778,0.351493,0.845082,3.847028,0.544379
4,201028,Cell_1,Voltron 2,0.750454,7.880892,0.804088,0.700855,5.588473,0.437181
5,201028,Cell_2,Voltron 2,0.681750,4.253009,0.814735,0.859425,3.929486,0.422558
6,201028,Cell_3,Voltron 2,0.639632,3.414511,0.750422,0.833970,4.341270,0.397631
7,201028,Cell_4,Voltron 2,0.615448,3.482160,0.771902,1.168270,5.100433,0.527317


In [6]:
df['Cell'] = ['Cell_1','Cell_2','Cell_3','Cell_4','Cell_1','Cell_2','Cell_3','Cell_4']

In [7]:
# df

### Load params from .mat file

In [8]:
pd.set_option("display.max_rows",50)
df

,Date,Cell,Sensor,tau_f_act,tau_s_act,tau_f_act_percent,tau_f_deact,tau_s_deact,tau_f_deact_percent
0,201027,Cell_1,Voltron,0.955512,4.032292,0.581620,1.161170,11.063581,0.587017
1,201027,Cell_2,Voltron,0.767470,2.468679,0.554365,1.261434,4.724830,0.605563
2,201027,Cell_3,Voltron,0.987813,4.335913,0.640978,1.242117,5.448495,0.730667
3,201027,Cell_4,Voltron,0.690084,2.192778,0.351493,0.845082,3.847028,0.544379
4,201028,Cell_1,Voltron 2,0.750454,7.880892,0.804088,0.700855,5.588473,0.437181
5,201028,Cell_2,Voltron 2,0.681750,4.253009,0.814735,0.859425,3.929486,0.422558
6,201028,Cell_3,Voltron 2,0.639632,3.414511,0.750422,0.833970,4.341270,0.397631
7,201028,Cell_4,Voltron 2,0.615448,3.482160,0.771902,1.168270,5.100433,0.527317


In [9]:
datePath = pb.getPathInfo(imgPath)
# datePath = ['2020.03.02_000']
# datePath = datePath[9:]
# datePath = ['2020.10.28_001','2020.10.28_001-varnam']
datePath = ['201020', '201021', '201022']
datePath


['201020', '201021', '201022']

In [10]:
sensors = {
    '201020':'Voltron-ST',
    '201021':'Voltron 2-ST',
    '201020':'Voltron-ST',
    
}

In [11]:
data_all={}
for param in(df.columns):
    data_all[param] = []
for date in datePath:
    cellPath = pb.getPathInfo(os.path.join(imgPath, date))
    for cell in(cellPath):
#         print(os.path.join(rawPath, date, plate, cell))
        matFiles = glob(os.path.join(imgPath, date, cell) + '/*.mat')
        if len(matFiles) < 1:
            print('bad trial: ' + '/' + date + '/' + cell)
            continue
        matFiles = sorted(matFiles)
        print(matFiles)
        data_all['Date'].append(date)
        data_all['Cell'].append(cell)
        tau_on_fast = []
        tau_on_slow = []
        tau_off_fast = []
        tau_off_slow = []
        tau_on_fast_percent = []

        for matFile in (matFiles):

            data = pb.get_data(matFile)

            tau_on_fast_single = data['tau_on_fast'][0]
            tau_on_slow_single = data['tau_on_slow'][0]
            tau_off_fast_single = data['tau_off_fast'][0]
            tau_off_slow_single = data['tau_off_slow'][0]
            tau_on_fast_percent_single = data['bkg_norm_ave_onfastpercent'][0]

            tau_on_fast.append(tau_on_fast_single)
            tau_on_slow.append(tau_on_slow_single)
            tau_off_fast.append(tau_off_fast_single)
            tau_off_slow.append(tau_off_slow_single)

        tau_on_fast = np.mean(tau_on_fast)
        tau_on_slow = np.mean(tau_on_slow)
        tau_off_fast = np.mean(tau_off_fast)
        tau_off_slow = np.mean(tau_off_slow)
        data_all['tau_f_act'].append(tau_on_fast)
        data_all['tau_s_act'].append(tau_on_slow)
        data_all['tau_f_deact'].append(tau_off_fast)
        data_all['tau_s_deact'].append(tau_off_slow)
                
print('Done!')


['/Volumes/genie/BenArthur/patch-screen/HCImage/201020/cell_1/trial_0002.mat', '/Volumes/genie/BenArthur/patch-screen/HCImage/201020/cell_1/trial_0003.mat', '/Volumes/genie/BenArthur/patch-screen/HCImage/201020/cell_1/trial_0004.mat', '/Volumes/genie/BenArthur/patch-screen/HCImage/201020/cell_1/trial_0005.mat']


KeyError: 'bkg_norm_ave_onfastpercent'

In [19]:
for k,v in data_all.items():
    print(k,len(v))

Date 8
Cell 8
Sensor 0
tau_f_act 8
tau_s_act 8
tau_f_act_percent 0
tau_f_deact 0
tau_s_deact 16
tau_f_deact_percent 0


In [17]:
df_new = pd.DataFrame(data_all, columns = list(df)) # columns specification keeps the order of columns; index specification makes sure dict keys are the columns 
df_new.shape

ValueError: arrays must all be same length

In [ ]:
df = df.append(df_new, ignore_index = True)
df.index = np.arange(len(df))
df.shape


In [ ]:
df.to_pickle(sumPath+'/Summary_VC_all.pkl')
# df.to_excel(sumPath + '/summary.xlsx')

In [ ]:
# df = df.drop(df.index[len(df)-1]) # Drop the last row
# df


In [ ]:
# Check the size for each column is the same
# i = 0
# for k, v in data_all.items():
#     print(k, len(v))
#     i+=1
